In [18]:
from aut import *
from pyspark.sql.functions import asc, countDistinct, first

In [19]:
images = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities") \
  .images() \
  .select(remove_prefix_www(extract_domain("url")).alias("domain"), "url", "md5")

In [20]:
images.show(5, True)

+--------------------+--------------------+--------------------+
|              domain|                 url|                 md5|
+--------------------+--------------------+--------------------+
|       geocities.com|http://geocities....|ce4c718e925105232...|
|       geocities.com|http://www.geocit...|f6b631a4db5f4c7a3...|
|visit.webhosting....|http://visit.webh...|4f59788bde58d15d5...|
|       geocities.com|http://www.geocit...|2677171223600bf34...|
|       geocities.com|http://geocities....|0a089830419a5c0ed...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [21]:
links = images.groupBy("md5") \
              .count() \
              .where(countDistinct("domain")>=2)

In [22]:
result = images.join(links, "md5") \
               .groupBy("domain", "md5") \
               .agg(first("url").alias("image_url")) \
               .orderBy(asc("md5"))

In [24]:
result.show(10, False)

+---------------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|domain                           |md5                             |image_url                                                                                                                                                     |
+---------------------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cmsimg.marshfieldnewsherald.com  |022ac0cde4560864fc99a3d9e5210f6b|http://cmsimg.marshfieldnewsherald.com/apps/pbcsi.dll/bilde?Avis=U0&Dato=20091026&Kategori=CWS01&Lopenr=910260803&Ref=PH&NewTbl=1&Itemnr=1&maxW=130&Border=0  |
|cmsimg.stevenspointjournal.com   |022ac0cde4560864fc99a3d9e5210f6b|http://cmsimg.steven